Installing required libraries for stepfunctions

In [2]:
 import sys
 !{sys.executable} -m pip install --upgrade pip
 !{sys.executable} -m pip install -qU awscli boto3 "sagemaker>=2.0.0"
 !{sys.executable} -m pip install -qU "stepfunctions>=2.0.0"
 !{sys.executable} -m pip show sagemaker stepfunctions

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 1.3.0 requires botocore<1.20.50,>=1.20.49, but you have botocore 1.26.10 which is incompatible.
Name: sagemaker
Version: 2.94.0
Summary: Open source library for training and deploying models on Amazon SageMaker.
Home-page: https://github.com/aws/sagemaker-python-sdk/
Author: Amazon Web Services
Author-email: 
License: Apache License 2.0
Location: /home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages
Requires: attrs, boto3, google-pasta, importlib-metadata, numpy, packaging, pandas, pathos, protobuf, protobuf3-to-dict, smdebug-rulesconfig
Required-by: stepfunctions
---
Name: stepfunctions
Version: 2.3.0
Summary: Open source library for developing data science workflows on AWS Step Functions.
Home-page: https://github.com/aws/aws-step-functions-data-science-sdk-python
Author: Amazon Web Servi

# 1. Setting up notebook with parameters and libraries

In [80]:
# importing Python libraies
import stepfunctions
import logging

from stepfunctions.steps import *
from stepfunctions.workflow import Workflow
from stepfunctions import steps
from stepfunctions.inputs import ExecutionInput
from sagemaker.processing import Processor,ProcessingInput, ProcessingOutput
import calendar
import time
import sagemaker
from sagemaker.inputs import TrainingInput
import boto3
from sagemaker.network import NetworkConfig

stepfunctions.set_stream_logger(level=logging.INFO)

### Defining paramters

This needs to be changed if we are taking it to different environment

In [139]:
v_workflow_execution_role = "arn:aws:iam::525102048888:role/poc-sagemaker-step-functi-MachineLearningWorkflowE-1XFI2UPRXFTXE" # Step function IAM role ARN
v_preprocessing_iam_role = "arn:aws:iam::525102048888:role/service-role/AmazonSageMaker-ExecutionRole-20191105T125227" # IAM role for preprocessing container
v_preprocessing_instance_type = "ml.m5.xlarge" # Instance type for preprocessing container it changes as per workload
v_s3_input_bucket = "wi-cred-datalake-dev-raw" # S3 bucket for input and output data
v_prefix_for_input_data = "transformed/monitoring/inbound/realtime/ll/2022/06/06/08/"  # Prefix where data is stored
groundtruth="transformed/monitoring/inbound/currentrun/groundtruth/realtime/ll/2022/06/06/07"
v_region = 'us-east-1' # AWS region
sec_groups = ["sg-044e0e7ce4f5721c0"]
subnets = ["subnet-0cf0e3f46326aa259",
           "subnet-0156b7f5500cf0b78",
           "subnet-032420199163cff9b"]
config_bucket = "wi-cred-datalake-dev-s3-mlops-config"

## 2. Defining preprocessing jobs

In [140]:
# Defining environment config for baseline jobs
environment = {
             'dataset_source': '/opt/ml/processing/input/endpoint/',
             'ground_truth_source': '/opt/ml/processing/input/groundtruth/',
             "output_path": "/opt/ml/processing/output",
              "publish_cloudwatch_metrics": "Disabled"              
            }

In [141]:
# Here we are creating baseline preprocesor
baseline_processor = Processor(image_uri='156813124566.dkr.ecr.us-east-1.amazonaws.com/sagemaker-model-monitor-groundtruth-merger',
                     role=v_preprocessing_iam_role,
                     instance_count=1,
                     instance_type=v_preprocessing_instance_type,
                    # network_config = NetworkConfig(security_group_ids = sec_groups, subnets = subnets),
                     env=environment)

In [142]:
input_data = "s3://{}/{}".format(v_s3_input_bucket,v_prefix_for_input_data)
gt_data="s3://{}/{}".format(v_s3_input_bucket,groundtruth)
inputs = [
    ProcessingInput(
        source=input_data, destination="/opt/ml/processing/input/endpoint/poc/AllTraffic/2022/05/25/13", input_name="input_data",
     ),
     ProcessingInput(
        source=gt_data, destination="/opt/ml/processing/input/groundtruth/gt_dataset_input", input_name="gt_data",
    )
]

outputs = [
    ProcessingOutput(
        source="/opt/ml/processing/output",
        destination="s3://{}/{}".format(v_s3_input_bucket,"transformed/monitoring/inbound/groundtruthmerge/ll"),
        output_name="gtmerged_data",
    )
]
gmt = time.gmtime()
ts = calendar.timegm(gmt)
baseline_name = "gtmerge-{}".format(ts)

In [143]:
print("s3://{}/{}".format(v_s3_input_bucket,"vehicle/usedcars/feature/lr/ModelDrift-BaselineOutput/"))


s3://wi-cred-datalake-dev-raw/vehicle/usedcars/feature/lr/ModelDrift-BaselineOutput/


In [144]:
gtmegre_preprocessing_step = steps.ProcessingStep(
    state_id='gtmerge', 
    processor=baseline_processor,
    job_name=baseline_name, 
    inputs=inputs, 
    #kms_key_id='3084dc48-1a82-435b-8a8d-8001f8890c08',
    outputs=outputs, 
    experiment_config=None, 
    wait_for_completion=True
)

## 3. Step Function

In [145]:
# First we chain the start pass state,preprocessing_step,
basic_path=Chain([gtmegre_preprocessing_step])

In [146]:
import time
print(time.localtime())

time.struct_time(tm_year=2022, tm_mon=6, tm_mday=8, tm_hour=7, tm_min=34, tm_sec=54, tm_wday=2, tm_yday=159, tm_isdst=0)


In [147]:
# Next, we define the workflow
import uuid
basic_workflow = Workflow(
    name="wi-mlops-modeldrift-gtmerge-job".format(
    uuid.uuid1().hex
) ,
    definition=basic_path,
    role=v_workflow_execution_role
)

#Render the workflow
basic_workflow.render_graph()

## 3.1 Create the workflow on AWS Step Functions

Create the workflow in AWS Step Functions with [create](https://aws-step-functions-data-science-sdk.readthedocs.io/en/latest/workflow.html#stepfunctions.workflow.Workflow.create).

In [148]:
basic_workflow.create()

[ERROR] A workflow with the same name already exists on AWS Step Functions. To update a workflow, use Workflow.update().


'arn:aws:states:us-east-1:525102048888:stateMachine:wi-mlops-modeldrift-gtmerge-job'

In [149]:
basic_workflow.update(definition=basic_workflow.definition,role=basic_workflow.role)

[INFO] Workflow updated successfully on AWS Step Functions. All execute() calls will use the updated definition and role within a few seconds. 


'arn:aws:states:us-east-1:525102048888:stateMachine:wi-mlops-modeldrift-gtmerge-job'

In [150]:
basic_workflow_execution = basic_workflow.execute(
    inputs={
    }
)

[INFO] Workflow execution started successfully on AWS Step Functions.


## 3.2 Review the execution progress

Render workflow progress with the [render_progress](https://aws-step-functions-data-science-sdk.readthedocs.io/en/latest/workflow.html#stepfunctions.workflow.Execution.render_progress).

This generates a snapshot of the current state of your workflow as it executes. This is a static image. Run the cell again to check progress. 

In [151]:
basic_workflow_execution.render_progress()

## 4 Downloading generated report on notebook

In [ ]:
# Downloading contraint file for evaluation
!aws s3 cp s3://$config_bucket/custommonitor/constraints.json .

In [ ]:
# Dowloading stats file for evaluation
!aws s3 cp s3://$config_bucket/custommonitor/statistics.json .

We can perform analysis on this file and put it on git hub

**Note :**
In order to refer it for modified location in monitoring schedule lambda function we need to changes the baseline stats location to prefix custom_monitoring/ from monitoring
